In [ ]:
from dataclasses import dataclass
from pathlib import Path

from pydantic_settings import BaseSettings


# 設定モデルを定義
class Settings(BaseSettings):
    GEMINI_API_KEY: str


settings = Settings()

In [ ]:
# Jupyter Notebook 内での非同期コードの実行を可能にする
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from pydantic import BaseModel
from pydantic_ai import Agent, BinaryContent, ModelSettings, NativeOutput
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.providers.google import GoogleProvider


class AgentInput(BaseModel):
    image_path: Path
    query: str


class AgentResponse(BaseModel):
    thinking_process: str
    fee: int


def run_agent(agent_input: AgentInput) -> AgentResponse:
    provider = GoogleProvider(api_key=settings.GEMINI_API_KEY)
    model = GoogleModel(
        "gemini-2.5-flash", settings=ModelSettings(temperature=0.0), provider=provider
    )
    agent = Agent(
        model,
        output_type=NativeOutput(AgentResponse),
        system_prompt=(
            "あなたは、コインパーキングの看板画像を読み取り、ユーザーの質問に答える有能なアシスタントです。\n"
            "看板に記載されている情報を読み取り、ユーザーの金額に関する質問に答えてください。"
        ),
    )
    image_path = agent_input.image_path
    query = agent_input.query
    with image_path.open("rb") as f:
        image_data = f.read()
    image_input = BinaryContent(data=image_data, media_type="image/jpeg")
    result = agent.run_sync([image_input, query])
    return result.output


In [ ]:
from pydantic_evals import Case

case1 = Case(
    name="weekday_2hours",
    inputs=AgentInput(
        image_path=Path("./data/sample.jpg"),
        query="平日の12時から14時までとめた時の料金はいくらですか？",
    ),
    expected_output=AgentResponse(
        thinking_process=(
            "平日は20分あたり330円です。"
            "12時から14時までは2時間（120分）なので、120 分 * 330  円 / 20 分 = 1980 円です。"
        ),
        fee=1980,
    ),
)

case2 = Case(
    name="saturday_6hours",
    inputs=AgentInput(
        image_path=Path("./data/sample.jpg"),
        query="土曜の18時から24時までとめた時の料金はいくらですか？",
    ),
    expected_output=AgentResponse(
        thinking_process=(
            "土曜日の18時から19時は、60 分 * 330 円 * / 20 分 =990円です。"
            "土曜日の19時から24時までは、5時間 * 60 分 * 330円 / 20分 = 4950円です。"
            "しかし最大料金の400円が適用されます。"
            "合計は990円 + 400円 = 1390円です。"
        ),
        fee=1390,
    ),
)

case3 = Case(
    name="weekday_to_holiday_overnight",
    inputs=AgentInput(
        image_path=Path("./data/sample.jpg"),
        query="木曜の21時から7時までとめた時の料金はいくらですか？なお翌日は祝日です。",
    ),
    expected_output=AgentResponse(
        thinking_process=(
            "木曜日の21時から翌日の7時までは、11 時間 * 60 分  * 330 円 / 20 分 = 10890 円です。"
            "しかし、最大料金の400円が適用されます。"
        ),
        fee=400,
    ),
)


In [ ]:
from pydantic_evals import Dataset
from pydantic_evals.evaluators import Evaluator, EvaluatorContext
from pydantic_evals.evaluators.common import IsInstance


@dataclass
class FeeMatch(Evaluator):
    def evaluate(self, ctx: EvaluatorContext) -> bool:
        output: AgentResponse = ctx.output
        expected: AgentResponse = ctx.expected_output
        return output.fee == expected.fee


dataset = Dataset(
    name="ParkingFeeDataset",
    cases=[case1, case2, case3],
    evaluators=[
        IsInstance(type_name="AgentResponse"),
        FeeMatch(),
    ],
)


In [ ]:
report = dataset.evaluate_sync(run_agent)
report.print(include_input=True, include_output=True)

Output()